# Install libraries 


In [1]:
import sys, os, json
import mne, sklearn, wandb
import numpy as np
import pandas as pd

from scipy.interpolate import interp1d
from nilearn import datasets, image, masking, plotting
from nilearn.input_data import NiftiLabelsMasker


# animation part
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt
# from celluloid import Camera   # it is convinient method to animate
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation


## torch libraries 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader, Subset

from pytorch_model_summary import summary


/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [2]:
%load_ext autoreload
%autoreload 2
sys.path.insert(1, os.path.realpath(os.path.pardir))

from utils import get_datasets
from utils import preproc
from utils import torch_dataset
from utils import train_utils
from utils import inference
from utils.models_arch import autoencoder_new, autoencoder_v3_separable, autoencoder_cross_corr

# Set all hyperparameters
- Cuda and GPU.
- Parameters of dataset. 
- random seed( if necessary). 


In [3]:
import random

torch.manual_seed(0)
random.seed(0)  # python operation seed
np.random.seed(0)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

print(torch.cuda.is_available(), torch.cuda.device_count())
torch.cuda.set_device(2)

True 4


In [4]:
config = dict(  
                dataset_name = 'CWL', # CWL
                new_fps=100, 
                freqs = np.logspace(np.log10(2), np.log10(99), 16), 
    
                n_channels = 30, # 30 
                n_roi = 6,
                
                bold_delay = 6,
                to_many = True,
                random_subsample = True,
                sample_per_epoch = 512, 
                WINDOW_SIZE = 2048,
                    
                optimizer='adamW',
                lr=3e-4,
                weight_decay=3e-4, # 3e-4
                batch_size=16, 
                
                preproc_type = 'dB_log',
                loss_function = 'mse_corr', 
                model_type = 'AE_CrossCorr_improved'
                )


hp_autoencoder = dict(n_electrodes=config['n_channels'],
                     n_freqs = len(config['freqs']),
                     n_channels_out=config['n_roi'],
                     
                     channels = [128, 128, 128, 64], 
                     kernel_sizes=[5, 5, 3], # kernel_sizes=[5, 5, 3]
                     strides=[8, 8, 4], # strides=[8, 8, 4]
                     dilation=[1, 1, 1], 
                     decoder_reduce=4, 
                     corr_proj_size = 64,
                     window_cross_corr=512)


config = {**hp_autoencoder, **config}

params_train = {'batch_size': config['batch_size'],
                'shuffle': True,
                'num_workers': 0}

params_val = {'batch_size': config['batch_size'],
              'shuffle': False}

# Upload preprocessed dataset from np files. 
It should accelerate speed of experiments.

In [5]:
with open("../data/processed/labels_roi_6.json", 'r') as f:
    labels_roi = json.load(f)


if config['dataset_name']=='CWL':
    dataset_path = '../data/processed/CWL/trio1_100_hz_6_roi_2_99_freqs.npz'
    
elif config['dataset_name']=='NODDI':
    dataset_path = '../data/processed/NODDI/32_100_hz_6_roi_2_99_freqs.npz'
else:
    print('no such dataset')


# download data
data = np.load(dataset_path)

train_dataset_prep = (data['x_train'], data['y_train'])
test_dataset_prep = (data['x_test'], data['y_test'])


# apply time dealy corrected
train_dataset_prep = preproc.bold_time_delay_align(train_dataset_prep, 
                                                   config['new_fps'],
                                                   config['bold_delay'])
test_dataset_prep = preproc.bold_time_delay_align(test_dataset_prep, 
                                                  config['new_fps'],
                                                  config['bold_delay'])


print('Size of train dataset:', train_dataset_prep[0].shape, train_dataset_prep[1].shape)
print('Size of test dataset:', test_dataset_prep[0].shape, test_dataset_prep[1].shape)

# torch dataset creation 
torch_dataset_train = torch_dataset.CreateDataset_eeg_fmri(train_dataset_prep, 
                                                            random_sample=config['random_subsample'], 
                                                            sample_per_epoch=config['sample_per_epoch'], 
                                                            to_many=config['to_many'], 
                                                            window_size = config['WINDOW_SIZE'])

torch_dataset_test = torch_dataset.CreateDataset_eeg_fmri(test_dataset_prep, 
                                                            random_sample=False, 
                                                            sample_per_epoch=None, 
                                                            to_many=config['to_many'], 
                                                            window_size = config['WINDOW_SIZE'])

# because you do not have strid for val data. 
torch_dataset_test = Subset(torch_dataset_test, np.arange(len(torch_dataset_test))[::100])

# init dataloaders for training
train_loader = torch.utils.data.DataLoader(torch_dataset_train, **params_train)
val_loader = torch.utils.data.DataLoader(torch_dataset_test, **params_val)




Size of train dataset: (30, 16, 20590) (6, 20590)
Size of test dataset: (30, 16, 5400) (6, 5400)


## Model investigation 

# Init Model, Loss, optimizers

In [6]:
model = autoencoder_cross_corr.CrossCorrAutoEncoder1D(**hp_autoencoder)

x = torch.zeros(4, 30, 16, 1024)
print('Input size: ', x.shape)
print('Output size: ',model(x).shape)
print('')

print(summary(model, x, show_input=True))

Channels for decoder [ 32  32  32 128]
Input size:  torch.Size([4, 30, 16, 1024])
Output size:  torch.Size([4, 6, 1024])

-----------------------------------------------------------------------
      Layer (type)         Input Shape         Param #     Tr. Param #
          Conv1d-1        [4, 6960, 2]         445,504         445,504
         Dropout-2          [4, 64, 2]               0               0
            ReLU-3          [4, 64, 2]               0               0
       ConvBlock-4      [4, 480, 1024]         184,576         184,576
       ConvBlock-5      [4, 128, 1024]          82,176          82,176
       ConvBlock-6       [4, 128, 128]          82,176          82,176
       ConvBlock-7        [4, 128, 16]          24,704          24,704
       ConvBlock-8         [4, 128, 4]          16,640          16,640
     UpConvBlock-9         [4, 128, 4]          12,352          12,352
    UpConvBlock-10         [4, 32, 16]           5,184           5,184
    UpConvBlock-11       

# Model training

In [ ]:
n_runs = 3

for i in range(n_runs):
    
    model = autoencoder_cross_corr.CrossCorrAutoEncoder1D(**hp_autoencoder)

    
    loss_func = train_utils.make_complex_loss_function(mse_weight = 0.1, 
                                                       corr_weight = 1,
                                                       manifold_weight = 0,
                                                       bound=1)
    train_step = train_utils.train_step

    optimizer = optim.Adam(model.parameters(),  # AdamW
                       lr=config['lr'], 
                       weight_decay=config['weight_decay'])
    
    
    parameters = {
        'EPOCHS': 750,
        'model': model, 
        'train_loader': train_loader, 
        'val_loader': val_loader, 
        'loss_function': loss_func,
        'train_step': train_step,
        'optimizer': optimizer, 
        'device': 'cuda', 
        'raw_test_data': test_dataset_prep,
        'show_info': 5, 
        'num_losses': 5,
        'labels': labels_roi,
        'inference_function': inference.model_inference_function, 
        'to_many': config['to_many']
    }



    path_to_save_wandb = 'common/koval_alvi/Checkpoints/wandb_brain'
    
    
    with wandb.init(project="eeg_fmri", config=config, save_code=True):
        
        wandb.define_metric("val/corr_mean", summary="max")

        if i == 0: 
            exp_name = wandb.run.name
        
        wandb.run.name = exp_name +'_run_' + str(i)
        
        print(config)
        print(parameters['model'])
        print(summary(model, torch.zeros(4, config['n_channels'],
                                         len(config['freqs']), config['WINDOW_SIZE']), show_input=True))
        
        model = train_utils.wanb_train_regression(**parameters)
        

Channels for decoder [ 32  32  32 128]


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: koval_alvi (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'n_electrodes': 30, 'n_freqs': 16, 'n_channels_out': 6, 'channels': [128, 128, 128, 64], 'kernel_sizes': [5, 5, 3], 'strides': [8, 8, 4], 'dilation': [1, 1, 1], 'decoder_reduce': 4, 'corr_proj_size': 64, 'window_cross_corr': 512, 'dataset_name': 'CWL', 'new_fps': 100, 'freqs': array([ 2.        ,  2.59420132,  3.36494024,  4.3646662 ,  5.6614114 ,
        7.34342046,  9.52515552, 12.3550855 , 16.02578954, 20.78706217,
       26.96291204, 34.97361097, 45.36429384, 58.84205542, 76.32406886,
       99.        ]), 'n_channels': 30, 'n_roi': 6, 'bold_delay': 6, 'to_many': True, 'random_subsample': True, 'sample_per_epoch': 512, 'WINDOW_SIZE': 2048, 'optimizer': 'adamW', 'lr': 0.0003, 'weight_decay': 0.0003, 'batch_size': 16, 'preproc_type': 'dB_log', 'loss_function': 'mse_corr', 'model_type': 'AE_CrossCorr_improved'}
CrossCorrAutoEncoder1D(
  (project): Sequential(
    (0): Conv1d(6960, 64, kernel_size=(1,), stride=(1,))
    (1): Dropout(p=0.25, inplace=False)
    (2): ReLU()
  )
  (spatia

wandb: Network error (ReadTimeout), entering retry loop.


..................................................................
Epoch 490 train loss_0 : -0.911 val loss_0 : -0.324 train loss_1 : 0.926 val loss_1 : 0.454 train loss_2 : 0.151 val loss_2 : 1.29 train loss_3 : 1.82 val loss_3 : 5.18 
................................................................................................................................................................
Epoch 495 train loss_0 : -0.91 val loss_0 : -0.24 train loss_1 : 0.925 val loss_1 : 0.382 train loss_2 : 0.149 val loss_2 : 1.42 train loss_3 : 1.83 val loss_3 : 5.36 
................................................................................................................................................................
Epoch 500 train loss_0 : -0.911 val loss_0 : -0.201 train loss_1 : 0.926 val loss_1 : 0.359 train loss_2 : 0.15 val loss_2 : 1.58 train loss_3 : 1.81 val loss_3 : 5.25 
........................................................................................................

...

Channels for decoder [ 32  32  32 128]


wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'n_electrodes': 30, 'n_freqs': 16, 'n_channels_out': 6, 'channels': [128, 128, 128, 64], 'kernel_sizes': [5, 5, 3], 'strides': [8, 8, 4], 'dilation': [1, 1, 1], 'decoder_reduce': 4, 'corr_proj_size': 64, 'window_cross_corr': 512, 'dataset_name': 'CWL', 'new_fps': 100, 'freqs': array([ 2.        ,  2.59420132,  3.36494024,  4.3646662 ,  5.6614114 ,
        7.34342046,  9.52515552, 12.3550855 , 16.02578954, 20.78706217,
       26.96291204, 34.97361097, 45.36429384, 58.84205542, 76.32406886,
       99.        ]), 'n_channels': 30, 'n_roi': 6, 'bold_delay': 6, 'to_many': True, 'random_subsample': True, 'sample_per_epoch': 512, 'WINDOW_SIZE': 2048, 'optimizer': 'adamW', 'lr': 0.0003, 'weight_decay': 0.0003, 'batch_size': 16, 'preproc_type': 'dB_log', 'loss_function': 'mse_corr', 'model_type': 'AE_CrossCorr_improved'}
CrossCorrAutoEncoder1D(
  (project): Sequential(
    (0): Conv1d(6960, 64, kernel_size=(1,), stride=(1,))
    (1): Dropout(p=0.25, inplace=False)
    (2): ReLU()
  )
  (spatia

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...............................................................
Epoch 170 train loss_0 : -0.899 val loss_0 : -0.293 train loss_1 : 0.917 val loss_1 : 0.428 train loss_2 : 0.171 val loss_2 : 1.35 train loss_3 : 2.07 val loss_3 : 5.7 
................................................................................................................................................................
Epoch 175 train loss_0 : -0.899 val loss_0 : -0.309 train loss_1 : 0.916 val loss_1 : 0.433 train loss_2 : 0.172 val loss_2 : 1.24 train loss_3 : 2.08 val loss_3 : 5.78 
.......................

# 